In [1]:
import sys
from PyQt5 import uic, QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import QApplication, QDialog, QMainWindow, QPushButton
import pandas as pd
import random
import import_ipynb
from Redapy_query import query_final
from Scrapeo_2017 import make_query, make_dataframe

BASE_URL = "https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"
QtWidgets.QApplication.setAttribute(QtCore.Qt.AA_EnableHighDpiScaling, True) #enable highdpi scaling
QtWidgets.QApplication.setAttribute(QtCore.Qt.AA_UseHighDpiPixmaps, True) #use highdpi icons

class Redapy_GUI(QMainWindow):
    
    def __init__(self):
        super().__init__()
        uic.loadUi("Redapy.ui", self)
              
        # sys.stdout = EmittingStream(textWritten=self.normalOutputWritten)

        # Desactiva Var2 cuando consulta es Frequency
        self.comboBox_consulta.currentIndexChanged.connect(self.on_change)
                        
        # Cargar datos cuando se selecciona BD
        self.comboBox_BD.currentTextChanged.connect(self.cargar_datos)
        self.comboBox_var1.setEditable(True)
        self.comboBox_var1.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_var1.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)
        self.comboBox_var2.setEditable(True)
        self.comboBox_var2.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_var2.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)
        self.comboBox_filtro.setEditable(True)
        self.comboBox_filtro.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_filtro.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)
        self.comboBox_seleccion_ubigeos.setEditable(True)
        self.comboBox_seleccion_ubigeos.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_seleccion_ubigeos.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)

       
        # boton Randomize
        self.Boton_Randomize.clicked.connect(self.randomize)
        
        # boton Limpiar
        self.Limpiar_Inputs.clicked.connect(self.reset_main_ui)
        
        # boton lineas_redatam
        self.Boton_Lineas_Redatam.clicked.connect(self.lineas_redatam)
        
        # boton Scrapeo
        self.Boton_Scrapear.clicked.connect(self.consulta)
        
    def on_change(self, newIndex):
        if newIndex==2:
            self.comboBox_var2.setEnabled(False)
            self.comboBox_var2.setCurrentIndex(0)
            self.comboBox_nivel_var2.setEnabled(False)
            self.comboBox_nivel_var2.setCurrentIndex(0)
        else: 
            self.comboBox_var2.setEnabled(True)
            self.comboBox_nivel_var2.setEnabled(True)
                 
    def cargar_datos(self, value):
        if self.comboBox_BD.currentText():
            datos_bd = self.comboBox_BD.currentText()
            datos_bd_ubigeos = pd.read_excel(datos_bd+'.xlsx',header=0, sheet_name=0, converters={'UBIGEO':str, 'NOMBRE':str})
            datos_bd_variables = pd.read_excel(datos_bd+'.xlsx',header=0, sheet_name=1, converters={'VAR':str, 'CATEGORIA':str})

            lista_ubigeos = list(set(datos_bd_ubigeos['UBIGEO'].tolist()))
            self.comboBox_seleccion_ubigeos.clear()
            self.comboBox_seleccion_ubigeos.addItems(lista_ubigeos)
        
            lista_variables = list(set(datos_bd_variables['VAR_COMPLETO'].tolist()))
            self.comboBox_filtro.clear()
            self.comboBox_filtro.addItems(lista_variables)
            self.comboBox_var1.clear()
            self.comboBox_var1.addItems(lista_variables)
            self.comboBox_var2.clear()
            self.comboBox_var2.addItems(lista_variables)
            global comboBox_consulta_max
            comboBox_consulta_max = self.comboBox_consulta.count()
            global comboBox_var1_max
            comboBox_var1_max = self.comboBox_var1.count()
            global comboBox_var2_max
            comboBox_var2_max = self.comboBox_var2.count()
            global comboBox_filtro_max
            comboBox_filtro_max = self.comboBox_filtro.count()
            global comboBox_seleccion_ubigeos 
            comboBox_seleccion_ubigeos = self.comboBox_seleccion_ubigeos.count()
        
    def randomize (self, value):
        self.comboBox_var1.setCurrentIndex(random.randint(0,comboBox_var1_max))
        self.comboBox_var2.setCurrentIndex(random.randint(0,comboBox_var2_max))
        self.comboBox_filtro.setCurrentIndex(random.randint(0,comboBox_filtro_max))
        self.comboBox_seleccion_ubigeos.setCurrentIndex(random.randint(0,comboBox_seleccion_ubigeos))
        self.comboBox_consulta.setCurrentIndex(random.randint(1,2))
        self.comboBox_var2.update()
        self.comboBox_nivel_var2.update()

    def reset_main_ui(self, value):
        self.comboBox_consulta.setCurrentIndex(0)
        self.comboBox_var1.setCurrentText("")
        self.comboBox_nivel_var1.setCurrentIndex(0)
        self.comboBox_var2.setCurrentText("")
        self.comboBox_nivel_var2.setCurrentIndex(0)
        self.comboBox_filtro.setCurrentText("")
        self.comboBox_nivel_filtro.setCurrentIndex(0)
        self.comboBox_seleccion_ubigeos.setCurrentText("")
        self.comboBox_seleccion_nivel.setCurrentIndex(0)
        self.comboBox_areabreak_nivel.setCurrentIndex(0)
        
    def lineas_redatam(self, value):
        tipo, variables1, variables2, selection, area_break = ([] for i in range(5))
        tipo = self.comboBox_consulta.currentText() if self.comboBox_consulta.currentText() else ""
        variables1.append(self.comboBox_var1.currentText()) if self.comboBox_var1.currentText() else ""
        variables2.append(self.comboBox_var2.currentText()) if self.comboBox_var2.currentText() else ""
        selection.append(self.comboBox_seleccion_ubigeos.currentText()) if self.comboBox_seleccion_ubigeos.currentText() else ""
        if selection:
            if len(selection[0]) == 2:
                selection[0] = "Departam "+selection[0]
            if len(selection[0]) == 4:
                selection[0] = "Provinci "+selection[0]
            if len(selection[0]) == 6:
                selection[0] = "Distrito "+selection[0]
        area_break.append(self.comboBox_areabreak_nivel.currentText()) if self.comboBox_areabreak_nivel.currentText() else ""
        universe_filter = "" 
        title = ""
        self.Lineas_Redatam.setText(query_final(tipo=tipo,variables1=variables1,variables2=variables2,selection=selection,area_break=area_break))
        
    def consulta(self, value):
        print(self.Lineas_Redatam.toPlainText())
        merged = pd.concat(make_dataframe(make_query(self.Lineas_Redatam.toPlainText(),BASE_URL)))
        # Excelwriter = pd.ExcelWriter("tablas.xlsx",engine="xlsxwriter") #exporta en excel dataframe final
        # merged.to_excel(Excelwriter)
        # Excelwriter.save()
        model = PandasModel(merged)
        self.tabla_resultados.setModel(model)
        self.tabla_resultados.show()
        
    def __del__(self):
        # Restore sys.stdout
        sys.stdout = sys.__stdout__

    def normalOutputWritten(self, text):
        """Append text to the QTextEdit."""
        # Maybe QTextEdit.append() works as well, but this is how I do it:
        cursor = self.textEdit.textCursor()
        cursor.movePosition(QtGui.QTextCursor.End)
        cursor.insertText(text)
        self.textEdit.setTextCursor(cursor)
        self.textEdit.ensureCursorVisible()
        
class PandasModel(QtCore.QAbstractTableModel):
    def __init__(self, data, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self._data = data

    def rowCount(self, parent=None):
        return len(self._data.values)

    def columnCount(self, parent=None):
        return self._data.columns.size

    def data(self, index, role=QtCore.Qt.DisplayRole):
        if index.isValid():
            if role == QtCore.Qt.DisplayRole:
                return QtCore.QVariant(str(
                    self._data.iloc[index.row()][index.column()]))
        return QtCore.QVariant()
    
class EmittingStream(QtCore.QObject):

    textWritten = QtCore.pyqtSignal(str)

    def write(self, text):
        self.textWritten.emit(str(text))

    
if __name__=='__main__':
    app = QtWidgets.QApplication(sys.argv)
    if app is None:
        app = QtGui.QApplication(sys.argv)
    GUI = Redapy_GUI()
    GUI.show()
    # sys.exit(app.exec_())
    app.exec_()
    del app

importing Jupyter notebook from Redapy_query.ipynb
importing Jupyter notebook from Scrapeo_2017.ipynb
